In [ ]:
# Installing Azure Command -Line Interface (CLI)
'''
Install az command extension 
cl: which az; az --version; az extension list; az extension add -n ml -y; 
az ml --help; az login ; az acount list -o table
pip isntall azrureml-core; python; import azureml; print(azureml)
'''

In [ ]:
# AzureMl Studio with Python

'''
From service principle: we need secret 

'''

In [ ]:
# Introduction: Automatio with Command-Line Tools
import os
import sys
import json
import argparse


sys.argv: # is a list or arguments pasted in the command line

parser=argparse.ArgumentParser(description="this is the jformat tool")
parser.add_argument("--sort", action="store_true", help="set the sorting")
args = parser.parse_args()
print(args.sort)

In [ ]:
# Using the Click Framework
import click

def formatter(string, sort_keys=True, indent=4):
    loaded_json = json.loads(string)
    return json.dumps(loaded_json, sort_keys=sort_keys, indent=indent)

@click.command()
@click.argument('path', type=click.Path(exists=True))
@click.option("--sort", "-s", is_flag=True)
def main(path, sort):
    with open(path, 'r') as _f:
        print(
            formatter(_f.read(), sort_keys=sort)
        )


if __name__=="__main__":
    main()

In [ ]:
# Packaging your project

from setuptools import setup, find_packages

setup(
    name=" aaa",
    version="0.01",
    description="Reformating files to stdout",
    install_requires=["click", "colorama"],
    entry_points= """
    [console_scripts]
    jformat=jformat.main:main
    """,
    author="Calvin",
    author_email="atewamba@gmail.com",
    packages=find_packages()
)


'''
cl: python setup.py develop
'''

In [ ]:
# Solving a Machine Learning Problem with CLI Tool



### Introduction to Flask

In [ ]:
#webapp/app.py
from flask import Flask, abort

app=Flask(__name__)

@app.route("/")
def two_hundred():
    return "<h1> 200! all is good from Duke University</h1>"

@app.route("/error")
def error():
    abort(500, "ooh some error!")


if __name__=="__main":
    app.run(debug=True, port=8000, host="0.0.0.0")

'''
On the terminal: create an environment, install the requirements, run: webapp/app.py

'''

In [ ]:
# Buiding an API with Flask
"""
webapp/app.py
"""

from flask import Flask, request, jsonify
import torch
import numpy as np
from transformers import RobertaTokenizer
import annxruntime

app=Flask(__name__)
tokenizer=RobertaTokenizer.from_pretrained("roberta-base")
session=onnxruntime.InferenceSession("roberta-sequence-classification-9.onnx")

def to_numpy(tensor):
    return (
        tensor.detach().cpu.numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )

@app.route("/")
def home():
    return "<h2> RoBERTa sentiment analysis </h2>"

@app.route("/predict", methods=["POST"])
def predict():
    inputs_ids = torch.tensor(
        tokenizer.encode(request.json[0], add_special_tokens=True)
    ).unsqueeze(
        0
    )

    inputs = {session.get_inputs()(0).name: to_numpy(inputs_ids)}
    out = session.run(None, inputs)

    result = np.argmax(out)

    return jsonify({"position": bool(result)})

if __name__=="__main__":
    app.run(host="0.0.0.0", port=5000, debug=True)

'''
cl: curl -x Post --header " Content-Type: application/json" --data '["using curl is not to my liking"]' http://127.0.0.1:5000/predict
'''



In [ ]:
# Introduction to the FastAPI

from datetime import datetime
from  os.path import dirname, abspath, join
from fastapi import FastAPI
from fastapi.responses import FileResponse
from fastapi.staticfiles import StaticFiles
from pydantic import BaseModel

current_dir = dirname(abspath(__file__))
static_path = join(current_dir, "static")

# Create the FastAPI application

app = FastAPI()

app.mount("/ui", StaticFiles(directory=static_path), name="ui")


class Body(BaseModel):
    strftime:str


@app.get("/")
def root():
    return  FileResponse("static/index.html")


@app.post('/generate')
def generate(body:Body):
    '''
    Generate the current time given s strftime template. Fro example:
    '%Y-%m-%dT%H:%M:%.%f'
    '''

    tmpl =body.strftime or '%Y-%m-%dT%H:%M:%.%f'
    return {'date': datetime.now().strftime(tmpl)}

'''
uvicorn --host 0.0.0.0 main:app
'''

In [ ]:
# Real word application with fastAPI

from transformers import pipeline
from fastapi import FastAPI, Response
from pydantic import BaseModel


generator = pipeline("text-generation", model="gpt2")

app = FastAPI()


class Body(BaseModel):
    text:str


@app.get('/')
def root():
    return Response("<h1> A self-documenting API to interact with GPT2 model and generate text </h1>")

@app.post("/genarate")
def predict(body:Body):
    results = generator(body.text, max_length=35, num_return_sequences=1)
    return results[0]


'''
cl: uvicon --host 0.0.0.0 main:app
'''

In [ ]:
# API Best Practices

'''
HTTP Error codes
400- bad request
401 - Unauthorize
404 Not found

Different types of request
GET: Read Only
POST: Write Only
PUT: Update existing
HEAD: Does it exist?

'''